# Following Pinecone Quickstart
* [https://app.pinecone.io/](https://app.pinecone.io/)

In [1]:
import openai
import os

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [4]:
from pinecone import Pinecone
import os, time

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "developer-quickstart-py"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model": "llama-text-embed-v2",
            "field_map": {"text": "chunk_text"}
        }
    )
    # optional: wait until the index is ready
    while pc.describe_index(index_name).status.get("ready") is False:
        time.sleep(5)


In [5]:
index = pc.Index(index_name)  

records = [
    { "_id": "rec1", "chunk_text": "The Eiffel Tower was completed in 1889 and stands in Paris, France.", "category": "history" },
    { "_id": "rec2", "chunk_text": "Photosynthesis allows plants to convert sunlight into energy.", "category": "science" },
    { "_id": "rec3", "chunk_text": "Albert Einstein developed the theory of relativity.", "category": "science" },
    { "_id": "rec4", "chunk_text": "The mitochondrion is often called the powerhouse of the cell.", "category": "biology" },
    { "_id": "rec5", "chunk_text": "Shakespeare wrote many famous plays, including Hamlet and Macbeth.", "category": "literature" },
    { "_id": "rec6", "chunk_text": "Water boils at 100°C under standard atmospheric pressure.", "category": "physics" },
    { "_id": "rec7", "chunk_text": "The Great Wall of China was built to protect against invasions.", "category": "history" },
    { "_id": "rec8", "chunk_text": "Honey never spoils due to its low moisture content and acidity.", "category": "food science" },
    { "_id": "rec9", "chunk_text": "The speed of light in a vacuum is approximately 299,792 km/s.", "category": "physics" },
    { "_id": "rec10", "chunk_text": "Newton's laws describe the motion of objects.", "category": "physics" },
    { "_id": "rec11", "chunk_text": "Shema is read two times a day, morning and evening.", "category": "talmud" }
]

index.upsert_records("ns1", records)

In [6]:
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'ns1': {'vector_count': 11}},
 'total_vector_count': 11,
 'vector_type': 'dense'}


In [7]:
query = "Famous historical structures and monuments"
query = "When to read Shema"
results = index.search(
    namespace="ns1",
    query={
        "top_k": 5,
        "inputs": {
            'text': query
        }
    }
)

print(results)

{'result': {'hits': [{'_id': 'rec11',
                      '_score': 0.5928104519844055,
                      'fields': {'category': 'talmud',
                                 'chunk_text': 'Shema is read two times a day, '
                                               'morning and evening.'}},
                     {'_id': 'rec10',
                      '_score': 0.011026806198060513,
                      'fields': {'category': 'physics',
                                 'chunk_text': "Newton's laws describe the "
                                               'motion of objects.'}},
                     {'_id': 'rec1',
                      '_score': 0.0003433480451349169,
                      'fields': {'category': 'history',
                                 'chunk_text': 'The Eiffel Tower was completed '
                                               'in 1889 and stands in Paris, '
                                               'France.'}},
                     {'_id': 'rec4',


In [8]:
reranked_results = index.search(
    namespace="ns1",
    query={
        "top_k": 5,
        "inputs": {
            'text': query
        }
    },
    rerank={
        "model": "bge-reranker-v2-m3",
        "top_n": 5,
        "rank_fields": ["chunk_text"]
    },
    fields=["category", "chunk_text"]
)

print(reranked_results)

{'result': {'hits': [{'_id': 'rec11',
                      '_score': 0.9873262047767639,
                      'fields': {'category': 'talmud',
                                 'chunk_text': 'Shema is read two times a day, '
                                               'morning and evening.'}},
                     {'_id': 'rec1',
                      '_score': 1.6314539607265033e-05,
                      'fields': {'category': 'history',
                                 'chunk_text': 'The Eiffel Tower was completed '
                                               'in 1889 and stands in Paris, '
                                               'France.'}},
                     {'_id': 'rec10',
                      '_score': 1.6187581422855146e-05,
                      'fields': {'category': 'physics',
                                 'chunk_text': "Newton's laws describe the "
                                               'motion of objects.'}},
                     {'_id': 'rec4